# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK, LinearAlgebra

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si, Si]
positions =  [ones(3)/8, -ones(3)/8]

# We take very (very) crude parameters
model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[1, 1, 1]);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, max_iter; tol)
    mixing_factor = .7
    x = x0
    fx = f(x)
    for n = 1:max_iter
        inc = fx - x
        if norm(inc) < tol
            break
        end
        x = x + mixing_factor * inc
        fx = f(x)
    end
    (fixpoint=x, converged=norm(fx-x) < tol)
end;

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (; λ, X, residual_norms=[], iterations=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.
In the mixing interface `δF` is $(ρ_\text{out} - ρ_\text{in})$, i.e.
the difference in density between two subsequent SCF steps and the `mix`
function returns $δρ$, which is added to $ρ_\text{in}$ to yield $ρ_\text{next}$,
the density for the next SCF step.

In [4]:
struct MyMixing
    n_simple  # Number of iterations for simple mixing
end
MyMixing() = MyMixing(2)

function DFTK.mix_density(mixing::MyMixing, basis, δF; n_iter, kwargs...)
    if n_iter <= mixing.n_simple
        return δF  # Simple mixing -> Do not modify update at all
    else
        # Use the default KerkerMixing from DFTK
        DFTK.mix_density(KerkerMixing(), basis, δF; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-8,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.224299812278                   -0.48    0.0
  2   -7.247842676288       -1.63       -0.87    0.0
  3   -7.251068302542       -2.49       -1.31    0.0
  4   -7.251272329754       -3.69       -1.62    0.0
  5   -7.251322045645       -4.30       -1.92    0.0
  6   -7.251334385000       -4.91       -2.22    0.0
  7   -7.251337572211       -5.50       -2.51    0.0
  8   -7.251338438752       -6.06       -2.79    0.0
  9   -7.251338687652       -6.60       -3.06    0.0
 10   -7.251338762977       -7.12       -3.32    0.0
 11   -7.251338786827       -7.62       -3.58    0.0
 12   -7.251338794658       -8.11       -3.83    0.0
 13   -7.251338797301       -8.58       -4.08    0.0
 14   -7.251338798212       -9.04       -4.31    0.0
 15   -7.251338798530       -9.50       -4.55    0.0
 16   -7.251338798642       -9.95       -4.78    0.0
 17   -7.251338798682      -10.40       -5.01 

Note that the default convergence criterion is the difference in
density. When this gets below `tol`, the
"driver" `self_consistent_field` artificially makes the fixed-point
solver think it's converged by forcing `f(x) = x`. You can customize
this with the `is_converged` keyword argument to
`self_consistent_field`.